필요한 라이브러리를 import해줍니다.

* Embedding을 통해 범주형 데이터를 벡터로 바꿔주겠습니다.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from tensorflow.keras.layers import Embedding 

상품 구매 목록에 대한 데이터가 다음과 같이 주어졌습니다.

In [2]:
data = pd.DataFrame({"product":['당근','토마토','샐러드','토마토','오이','소고기','닭가슴살','토마토','샐러드','계란','두부','닭가슴살'], "num":[3,6,1,2,3,2,10,4,1,10,2,10]})
data

,product,num
0,당근,3
1,토마토,6
2,샐러드,1
3,토마토,2
4,오이,3
5,소고기,2
6,닭가슴살,10
7,토마토,4
8,샐러드,1
9,계란,10


위 데이터 중 "product"컬럼에 해당하는 상품 품목들은 범주형으로, 인코딩해야합니다.

토마토, 샐러드, 닭가슴살은 중복 구매된 상품입니다.

사전을 만들기 위해 중복을 제외한 상품 목록을 추출합니다

총 8가지의 상품이 있습니다.

In [3]:
product_list = list(set(data["product"]))
print(product_list)

['소고기', '오이', '샐러드', '토마토', '닭가슴살', '두부', '계란', '당근']


중복을 제외한 product_list를 인덱스로 변환하기 위해 vocab을 만들어줍니다

In [4]:
product_vocab = {name:i for i,name in enumerate(product_list)}

print(product_vocab)

{'소고기': 0, '오이': 1, '샐러드': 2, '토마토': 3, '닭가슴살': 4, '두부': 5, '계란': 6, '당근': 7}


# one-hot encoding


[tf.one_hot](https://www.tensorflow.org/api_docs/python/tf/one_hot) 을 이용하면 원핫 인코딩 벡터를 만들어줍니다.

aurgument로는 변환하려는 데이터의 vocab 인덱스와, 전체 범주 길이를 넣어주면 됩니다.

In [5]:
print(  tf.one_hot(product_vocab["당근"], len(product_vocab))  )

tf.Tensor([0. 0. 0. 0. 0. 0. 0. 1.], shape=(8,), dtype=float32)


기존의 데이터를 다음과 같이 원핫 인코딩 벡터로 만들 수 있습니다

In [6]:
for index, row in data.iterrows():
    print(row['product'], row['num'], " \t--> ", tf.one_hot(product_vocab[row['product']], len(product_vocab)).numpy(), row['num']) 

당근 3  	-->  [0. 0. 0. 0. 0. 0. 0. 1.] 3
토마토 6  	-->  [0. 0. 0. 1. 0. 0. 0. 0.] 6
샐러드 1  	-->  [0. 0. 1. 0. 0. 0. 0. 0.] 1
토마토 2  	-->  [0. 0. 0. 1. 0. 0. 0. 0.] 2
오이 3  	-->  [0. 1. 0. 0. 0. 0. 0. 0.] 3
소고기 2  	-->  [1. 0. 0. 0. 0. 0. 0. 0.] 2
닭가슴살 10  	-->  [0. 0. 0. 0. 1. 0. 0. 0.] 10
토마토 4  	-->  [0. 0. 0. 1. 0. 0. 0. 0.] 4
샐러드 1  	-->  [0. 0. 1. 0. 0. 0. 0. 0.] 1
계란 10  	-->  [0. 0. 0. 0. 0. 0. 1. 0.] 10
두부 2  	-->  [0. 0. 0. 0. 0. 1. 0. 0.] 2
닭가슴살 10  	-->  [0. 0. 0. 0. 1. 0. 0. 0.] 10


# Embedding


[Embedding layer](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) 를 이용하면 데이터를 지정한 차원의 벡터로 만들어줍니다.

argument로는 전체 범주의 길이와 변환하려는 벡터의 길이를 넣어주고,

입력으로는 변환하려는 데이터의 vocab 인덱스를 넣어주면 됩니다.

이 경우 각 상품을 길이 3의 벡터로 만들겠습니다.

In [7]:
embedding_layer = Embedding(len(product_vocab), 3)
result = embedding_layer(product_vocab["당근"])
print(result) 

tf.Tensor([ 0.04321284 -0.01696879 -0.00422965], shape=(3,), dtype=float32)


기존의 데이터를 다음과 같이 임베딩 벡터로 만들 수 있습니다

In [8]:
for index, row in data.iterrows():
    print(row['product'], row['num'], " \t--> ", embedding_layer(product_vocab[row['product']]).numpy(), row['num']) 

당근 3  	-->  [ 0.04321284 -0.01696879 -0.00422965] 3
토마토 6  	-->  [-0.01854287  0.006738   -0.01718505] 6
샐러드 1  	-->  [-0.02538518  0.03959824  0.0391911 ] 1
토마토 2  	-->  [-0.01854287  0.006738   -0.01718505] 2
오이 3  	-->  [ 0.04307355 -0.01830908  0.01779013] 3
소고기 2  	-->  [ 0.03576297 -0.03392136  0.0140285 ] 2
닭가슴살 10  	-->  [-0.01698168  0.03696792  0.03616935] 10
토마토 4  	-->  [-0.01854287  0.006738   -0.01718505] 4
샐러드 1  	-->  [-0.02538518  0.03959824  0.0391911 ] 1
계란 10  	-->  [0.04175324 0.04546375 0.03643881] 10
두부 2  	-->  [-2.1994114e-05 -2.2459567e-02 -3.3894945e-02] 2
닭가슴살 10  	-->  [-0.01698168  0.03696792  0.03616935] 10


임베딩 레이어에는 인덱스로 변환된 데이터를 한번에 넣어 결과로 추론할 수 있습니다.

In [9]:
prod_idxs = np.array([product_vocab[prod] for prod in data["product"]])
print("▶데이터의 상품을 인덱스로 변환 : \n >> ", prod_idxs)
print()
print("▶데이터의 상품을 임베딩한 결과 shape : \n >> (데이터길이, 임베딩길이): ", embedding_layer(prod_idxs).shape)
print()
print("▶데이터의 상품을 임베딩한 결과 : \n >> ", embedding_layer(prod_idxs))


▶데이터의 상품을 인덱스로 변환 : 
 >>  [7 3 2 3 1 0 4 3 2 6 5 4]

▶데이터의 상품을 임베딩한 결과 shape : 
 >> (데이터길이, 임베딩길이):  (12, 3)

▶데이터의 상품을 임베딩한 결과 : 
 >>  tf.Tensor(
[[ 4.3212842e-02 -1.6968787e-02 -4.2296536e-03]
 [-1.8542875e-02  6.7379959e-03 -1.7185055e-02]
 [-2.5385177e-02  3.9598238e-02  3.9191101e-02]
 [-1.8542875e-02  6.7379959e-03 -1.7185055e-02]
 [ 4.3073546e-02 -1.8309079e-02  1.7790128e-02]
 [ 3.5762969e-02 -3.3921361e-02  1.4028501e-02]
 [-1.6981684e-02  3.6967922e-02  3.6169354e-02]
 [-1.8542875e-02  6.7379959e-03 -1.7185055e-02]
 [-2.5385177e-02  3.9598238e-02  3.9191101e-02]
 [ 4.1753236e-02  4.5463752e-02  3.6438812e-02]
 [-2.1994114e-05 -2.2459567e-02 -3.3894945e-02]
 [-1.6981684e-02  3.6967922e-02  3.6169354e-02]], shape=(12, 3), dtype=float32)
